In [53]:
# Import the necessary modules
import json
import requests
import re  # Import regular expressions
import boto3
import re  # Import regular expressions
from botocore.exceptions import NoCredentialsError, PartialCredentialsError
from rich import print as rprint
import pandas as pd
from decimal import Decimal

In [59]:
genesis_url = 'https://www-genesis.destatis.de/genesisWS/rest/2020/data/cube?'
user_id = 'DE17T29R57'
password = '4Bf/3Ap)3]r2,,h'

# Wärmepreisindex
cubecode = '61111BM006'
areatype = 'all'
category = 'all'
content = ''
start_year = ''
classifier1 = 'CC13B1'
key1 = 'CC13-77'
lang = 'en'

# Destatis Base Url
url = (f"{genesis_url}username={user_id}&password={password}&language=de&name={cubecode}&area={areatype}"
       f"&compress=true&contents={content}&startyear={start_year}")

# Add Classifyers to url
if classifier1:
       url += f'&classifyingvariable1={classifier1}'    
if key1:
       url += f'&classifyingkey1={key1}'

# request 
response = requests.get(url)

# First, parse the JSON response with Decimal conversion for floats
#Float types are not supported with dynamodb; use Decimal types instead
parsed_json = json.loads(response.text, parse_float=Decimal)

# Now, access the 'Content' part and split it by newline
data = parsed_json["Object"]['Content'].split('\n')


# Compile a regex pattern to find sequences of digits ending with "=100"
info_pattern = re.compile(r'\d+=100')

# Extract only the parts of lines that match the pattern "*=100"
info_matches = []
for line in data:
    matches = info_pattern.findall(line)  # find all matches in a line
    info_matches.extend(matches)  # add found matches to the list

# Convert info_matches to a single string to append to each filtered line
info_matches_str = ', '.join(info_matches)

# Filter out the relevant lines and append the matches string to each
filtered_response = [line + " " + info_matches_str for line in data if 'D' in line and 'e' in line and ('MONAT' in line or 'QUART' in line)]

# Split each line into its components and extract the relevant information
data = [{'field_D': parts[0],
         'field_DG': parts[1],
         'classifyingkey1': parts[2],
         'period': parts[3],
         'year': int(parts[4]),
         'value': float(parts[5]),
         'field_e': parts[6],
         'base': info_matches_str
        }
        for parts in (line.split(';') for line in filtered_response)]

# Create a DataFrame from the extracted data
df = pd.DataFrame(data)

# Determine if the data is monthly or quarterly based on the unique values in the 'period' column
if df['period'].str.contains('Q').any():
    period_key = 'quarter'
else:
    period_key = 'month'

# Create result dict
result = {'cubeCode': cubecode, 'content': content, 'classifyingVar1': classifier1, 'classifyingKey1': key1, 'data': []}
for year, group in df.groupby('year'):
    year_data = {
        'year': year,
        'df': [{
            period_key: row['period'],
            'value': row['value'],
            'base': row['base']
            } for _, row in group.iterrows()]
    }
    result['data'].append(year_data)
    
j = json.dumps(result, indent=4)
rprint(j)

{
    "cubeCode": "61111BM006",
    "content": "",
    "classifyingVar1": "CC13B1",
    "classifyingKey1": "CC13-77",
    "data": [
        {
            "year": 1991,
            "df": [
                {
                    "month": "MONAT01",
                    "value": 38.7,
                    "base": "2020=100"
                },
                {
                    "month": "MONAT02",
                    "value": 38.9,
                    "base": "2020=100"
                },
                {
                    "month": "MONAT03",
                    "value": 39.6,
                    "base": "2020=100"
                },
                {
                    "month": "MONAT04",
                    "value": 39.8,
                    "base": "2020=100"
                },
                {
                    "month": "MONAT05",
                    "value": 40.0,
                    "base": "2020=100"
                },
                {
                    "month": "MONAT06",
                    "value": 40.3,
                    "base": "2020=100"
                },
                {
                    "month": "MONAT07",
                    "value": 40.6,
                    "base": "2020=100"
                },
                {
                    "month": "MONAT08",
                    "value": 40.6,
                    "base": "2020=100"
                },
                {
                    "month": "MONAT09",
                    "value": 41.0,
                    "base": "2020=100"
                },
                {
                    "month": "MONAT10",
                    "value": 53.0,
                    "base": "2020=100"
                },
                {
                    "month": "MONAT11",
                    "value": 52.7,
                    "base": "2020=100"
                },
                {
                    "month": "MONAT12",
                    "value": 52.7,
                    "base": "2020=100"
                }
            ]
        },
        {
            "year": 1992,
            "df": [
                {
                    "month": "MONAT01",
                    "value": 52.6,
                    "base": "2020=100"
                },
                {
                    "month": "MONAT02",
                    "value": 52.4,
                    "base": "2020=100"
                },
                {
                    "month": "MONAT03",
                    "value": 51.9,
                    "base": "2020=100"
                },
                {
                    "month": "MONAT04",
                    "value": 52.0,
                    "base": "2020=100"
                },
                {
                    "month": "MONAT05",
                    "value": 52.0,
                    "base": "2020=100"
                },
                {
                    "month": "MONAT06",
                    "value": 52.2,
                    "base": "2020=100"
                },
                {
                    "month": "MONAT07",
                    "value": 52.4,
                    "base": "2020=100"
                },
                {
                    "month": "MONAT08",
                    "value": 52.4,
                    "base": "2020=100"
                },
                {
                    "month": "MONAT09",
                    "value": 52.4,
                    "base": "2020=100"
                },
                {
                    "month": "MONAT10",
                    "value": 52.0,
                    "base": "2020=100"
                },
                {
                    "month": "MONAT11",
                    "value": 51.8,
                    "base": "2020=100"
                },
                {
                    "month": "MONAT12",
                    "value": 51.8,
                    "base": "2020=100"
                }
            ]
        },
        {
            "year": 1993,
            "df": [
                {
          

In [68]:
# DynamoDB Configurations
table_name = 'onetable'
dynamodb = boto3.resource('dynamodb', region_name='eu-west-1')
table = dynamodb.Table(table_name)

In [69]:
# Function to upload data to DynamoDB
def upload_to_dynamodb(table, data_dict):
    try:
        for year_info in data_dict['data']:  # Iterate over each year
            year = year_info['year']
            for month_data in year_info['df']:  # Iterate over each month's data
                month_value = month_data['month']  # Extracting the month value from each month_data entry
                index = "index"
                sort_key = f"IND#{data_dict['cubeCode']}#{year}#{month_value}"  # Append month_value to sort key

                # Prepare the item dictionary for DynamoDB
                item = {
                    'pk': index,
                    'sk': sort_key,
                    'lsi1': data_dict['cubeCode'],
                    'lsi2': data_dict['classifyingVar1'],
                    'attribute1': data_dict['classifyingKey1'],
                    'attribute2': year,
                    'attribute3': month_data[period_key],  # Use period_key for the month column name
                    'attribute4': Decimal(str(month_data['value'])),  # Convert value to Decimal
                    'attribute5': month_data['base'],
                }

                # Put the item into DynamoDB
                table.put_item(Item=item)
                print(f"Uploaded item: {item}")

    except Exception as e:
        print(f"An error occurred: {e}")

# Assuming 'result' is your previously defined dictionary with data
upload_to_dynamodb(table, result)

Uploaded item: {'pk': 'index', 'sk': 'IND#61111BM006#1991#MONAT01', 'lsi1': '61111BM006', 'lsi2': 'CC13B1', 'attribute1': 'CC13-77', 'attribute2': 1991, 'attribute3': 'MONAT01', 'attribute4': Decimal('38.7'), 'attribute5': '2020=100'}
Uploaded item: {'pk': 'index', 'sk': 'IND#61111BM006#1991#MONAT02', 'lsi1': '61111BM006', 'lsi2': 'CC13B1', 'attribute1': 'CC13-77', 'attribute2': 1991, 'attribute3': 'MONAT02', 'attribute4': Decimal('38.9'), 'attribute5': '2020=100'}
Uploaded item: {'pk': 'index', 'sk': 'IND#61111BM006#1991#MONAT03', 'lsi1': '61111BM006', 'lsi2': 'CC13B1', 'attribute1': 'CC13-77', 'attribute2': 1991, 'attribute3': 'MONAT03', 'attribute4': Decimal('39.6'), 'attribute5': '2020=100'}
Uploaded item: {'pk': 'index', 'sk': 'IND#61111BM006#1991#MONAT04', 'lsi1': '61111BM006', 'lsi2': 'CC13B1', 'attribute1': 'CC13-77', 'attribute2': 1991, 'attribute3': 'MONAT04', 'attribute4': Decimal('39.8'), 'attribute5': '2020=100'}
Uploaded item: {'pk': 'index', 'sk': 'IND#61111BM006#1991#MO

In [71]:
# use PartiQL to query the table
# this solution avoids a potential SQL injection problem

# Initialize the DynamoDB service resource
dynamodb = boto3.client('dynamodb', region_name='eu-west-1') 

pk = 'index'
sk = 'IND#61111BM006#2020'# This should be the value of the Index that corresponds to the data you want

# The year you are interested in
year = "1995"

# Prepare the PartiQL statement to query data
stmt = f"SELECT * FROM {table_name} WHERE pk = ? AND begins_with(sk, ?)"

# Parameters for the query
pmt = [
    {"S": pk},
    {"S": sk}
]

# Execute the statement
resp = dynamodb.execute_statement(
    Statement=stmt , Parameters= pmt
)

# Print the response items
rprint(resp['Items'])

[
    {
        'lsi1': {'S': '61111BM006'},
        'lsi2': {'S': 'CC13B1'},
        'attribute1': {'S': 'CC13-77'},
        'attribute2': {'N': '2020'},
        'attribute5': {'S': '2020=100'},
        'sk': {'S': 'IND#61111BM006#2020'},
        'attribute3': {'S': 'MONAT12'},
        'attribute4': {'N': '96'},
        'pk': {'S': 'index'}
    },
    {
        'lsi1': {'S': '61111BM006'},
        'lsi2': {'S': 'CC13B1'},
        'attribute1': {'S': 'CC13-77'},
        'attribute2': {'N': '2020'},
        'attribute5': {'S': '2020=100'},
        'sk': {'S': 'IND#61111BM006#2020#MONAT01'},
        'attribute3': {'S': 'MONAT01'},
        'attribute4': {'N': '102.4'},
        'pk': {'S': 'index'}
    },
    {
        'lsi1': {'S': '61111BM006'},
        'lsi2': {'S': 'CC13B1'},
        'attribute1': {'S': 'CC13-77'},
        'attribute2': {'N': '2020'},
        'attribute5': {'S': '2020=100'},
        'sk': {'S': 'IND#61111BM006#2020#MONAT02'},
        'attribute3': {'S': 'MONAT02'},
        'attribute4': {'N': '102.5'},
        'pk': {'S': 'index'}
    },
    {
        'lsi1': {'S': '61111BM006'},
        'lsi2': {'S': 'CC13B1'},
        'attribute1': {'S': 'CC13-77'},
        'attribute2': {'N': '2020'},
        'attribute5': {'S': '2020=100'},
        'sk': {'S': 'IND#61111BM006#2020#MONAT03'},
        'attribute3': {'S': 'MONAT03'},
        'attribute4': {'N': '102.4'},
        'pk': {'S': 'index'}
    },
    {
        'lsi1': {'S': '61111BM006'},
        'lsi2': {'S': 'CC13B1'},
        'attribute1': {'S': 'CC13-77'},
        'attribute2': {'N': '2020'},
        'attribute5': {'S': '2020=100'},
        'sk': {'S': 'IND#61111BM006#2020#MONAT04'},
        'attribute3': {'S': 'MONAT04'},
        'attribute4': {'N': '102'},
        'pk': {'S': 'index'}
    },
    {
        'lsi1': {'S': '61111BM006'},
        'lsi2': {'S': 'CC13B1'},
        'attribute1': {'S': 'CC13-77'},
        'attribute2': {'N': '2020'},
        'attribute5': {'S': '2020=100'},
        'sk': {'S': 'IND#61111BM006#2020#MONAT05'},
        'attribute3': {'S': 'MONAT05'},
        'attribute4': {'N': '101.7'},
        'pk': {'S': 'index'}
    },
    {
        'lsi1': {'S': '61111BM006'},
        'lsi2': {'S': 'CC13B1'},
        'attribute1': {'S': 'CC13-77'},
        'attribute2': {'N': '2020'},
        'attribute5': {'S': '2020=100'},
        'sk': {'S': 'IND#61111BM006#2020#MONAT06'},
        'attribute3': {'S': 'MONAT06'},
        'attribute4': {'N': '101.1'},
        'pk': {'S': 'index'}
    },
    {
        'lsi1': {'S': '61111BM006'},
        'lsi2': {'S': 'CC13B1'},
        'attribute1': {'S': 'CC13-77'},
        'attribute2': {'N': '2020'},
        'attribute5': {'S': '2020=100'},
        'sk': {'S': 'IND#61111BM006#2020#MONAT07'},
        'attribute3': {'S': 'MONAT07'},
        'attribute4': {'N': '99.8'},
        'pk': {'S': 'index'}
    },
    {
        'lsi1': {'S': '61111BM006'},
        'lsi2': {'S': 'CC13B1'},
        'attribute1': {'S': 'CC13-77'},
        'attribute2': {'N': '2020'},
        'attribute5': {'S': '2020=100'},
        'sk': {'S': 'IND#61111BM006#2020#MONAT08'},
        'attribute3': {'S': 'MONAT08'},
        'attribute4': {'N': '99.2'},
        'pk': {'S': 'index'}
    },
    {
        'lsi1': {'S': '61111BM006'},
        'lsi2': {'S': 'CC13B1'},
        'attribute1': {'S': 'CC13-77'},
        'attribute2': {'N': '2020'},
        'attribute5': {'S': '2020=100'},
        'sk': {'S': 'IND#61111BM006#2020#MONAT09'},
        'attribute3': {'S': 'MONAT09'},
        'attribute4': {'N': '98.6'},
        'pk': {'S': 'index'}
    },
    {
        'lsi1': {'S': '61111BM006'},
        'lsi2': {'S': 'CC13B1'},
        'attribute1': {'S': 'CC13-77'},
        'attribute2': {'N': '2020'},
        'attribute5': {'S': '2020=100'},
        'sk': {'S': 'IND#61111BM006#2020#MONAT10'},
        'attribute3': {'S': 'MONAT10'},
        'attribute4': {'N': '97.5'},
        'pk': {'S': 'index'}
    },
    {
        'lsi1': {'S': '61111BM006'},
        'lsi